## Project Details

Module Name: CT5228
Assignmet 3: BER Homes Analysis
Name: Michael Lynagh
ID: 23202058
Year 1 - Autumn Semester
Date: 14/12/2025

In [11]:
import pandas as pd 
import numpy as np 
from sklearn.impute import SimpleImputer

In [7]:
# Step 1 - Import Data 
df = pd.read_csv('BER_Train.csv', encoding='utf-8')
df.head()

,CountyName,DwellingTypeDescr,Year_of_Construction,EnergyRating,GroundFloorArea(sq m),UValueWall,UValueRoof,UValueFloor,UValueWindow,UvalueDoor,...,WaterStorageVolume,DeclaredLossFactor,TempFactorUnadj,TempFactorMultiplier,GroundFloorArea,GroundFloorHeight,FirstFloorArea,FirstFloorHeight,GlazingPercent,Volume
0,Co. Wexford,Detached house,2004,D2,103.44,0.55,0.26,0.47,1.9,3.0,...,98.0,0.00,0.6,1.3,51.72,2.46,51.72,2.73,0.114906,536.8536
1,Co. Dublin,Semi-detached house,2017,A3,115.96,0.21,0.12,0.14,1.4,1.4,...,300.0,0.00,0.6,0.9,62.93,2.45,53.03,2.70,0.153989,597.1940
2,Co. Cork,Semi-detached house,2006,B2,123.08,0.22,0.13,0.37,1.4,3.0,...,87.0,1.38,0.6,0.9,62.41,2.46,60.67,2.74,0.182280,640.0160
3,Dublin 5,Semi-detached house,1966,D2,139.60,0.33,0.88,0.59,3.1,3.0,...,115.0,0.00,0.6,1.3,80.06,2.52,59.54,2.68,0.245111,725.9200
4,Co. Cork,Detached house,2016,A1,158.42,0.27,0.13,0.17,2.0,3.0,...,NaN,NaN,NaN,NaN,158.42,2.65,0.00,0.00,0.193304,419.8130


working steps 
Preprocessing
Handle missing values and encode categorical features if needed.
Apply scaling/normalisation to numerical features if needed.
Clearly state which preprocessing steps you used.

In [8]:

# Step 2 - Preprocessing 
# Calculate rows with missing data 
df.isna().sum()
missing_rows = df[df.isna().any(axis=1)]
len(missing_rows), missing_rows.index[:10].to_list()

(1875, [4, 6, 7, 8, 13, 15, 16, 18, 19, 20])

In [12]:
# Step 3 - Account for missing data 
numeric_cols = df.select_dtypes(include=np.number).columns.to_list()
columns_for_exclusion = ['CountyName', 'DwellingTypeDescr']
columns_for_inclusion = [col for col in numeric_cols if col not in columns_for_exclusion]
df_imputed = df.copy()

# Step 4 - Initialise the imputer, strategy for dealing with null values
imputer = SimpleImputer(strategy='median')

df_imputed[columns_for_inclusion] = imputer.fit_transform(df_imputed[columns_for_inclusion])

# Check these results 
print("Missing values after imputation")
print(df_imputed[columns_for_inclusion].isna().sum())
print(f"\nResults after imputation: {df_imputed.shape}")


Missing values after imputation
Year_of_Construction         0
GroundFloorArea(sq m)        0
UValueWall                   0
UValueRoof                   0
UValueFloor                  0
UValueWindow                 0
UvalueDoor                   0
WallArea                     0
RoofArea                     0
FloorArea                    0
WindowArea                   0
DoorArea                     0
NoStoreys                    0
CO2Rating                    0
HSMainSystemEfficiency       0
TGDLEdition                  0
MPCDERValue                  0
HSEffAdjFactor               0
HSSupplHeatFraction          0
HSSupplSystemEff             0
WHMainSystemEff              0
WHEffAdjFactor               0
SupplSHFuel                  0
SupplWHFuel                  0
SHRenewableResources         0
WHRenewableResources         0
NoOfChimneys                 0
NoOfOpenFlues                0
NoOfFansAndVents             0
NoOfFluelessGasFires         0
FanPowerManuDeclaredValue    0
HeatExc